In [1]:
# Load Kaggle API Token
from google.colab import files
!pip install -q kaggle > /dev/null
uploaded = files.upload()
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Load Data
!kaggle competitions download -c tensorflow-speech-recognition-challenge > /dev/null

# Unzip Data
!apt-get install p7zip-full > /dev/null
!p7zip -d train.7z > /dev/null
# !p7zip -d test.7z > /dev/null


Saving kaggle.json to kaggle.json
100% 501k/501k [00:00<00:00, 34.2MB/s]
100% 50.0/50.0 [00:00<00:00, 47.6kB/s]
100% 2.46G/2.46G [00:37<00:00, 69.5MB/s]
100% 1.04G/1.04G [00:23<00:00, 48.2MB/s]


In [14]:
from scipy.io import wavfile
from IPython.display import Audio
from os import walk
from matplotlib import pyplot as plt

debug = True

train_audio_path = '/content/train/audio/'

# Load all filenames into a dictionary so we can call on them easily
files = {}
for (dirpath, dirnames, filenames) in walk(train_audio_path):
  files[dirpath[21:]] = filenames

if debug:
  # Look at a small part of the dictionary
  for label, samples in files.items():
    print(label, '\t', samples[0:5])

 	 []
five 	 ['38d78313_nohash_4.wav', '8fe52b97_nohash_1.wav', 'f804cbb3_nohash_0.wav', '833a0279_nohash_2.wav', 'd85270c1_nohash_2.wav']
marvin 	 ['bed06fac_nohash_1.wav', '8fe52b97_nohash_1.wav', '7be9f58a_nohash_0.wav', '27c30960_nohash_0.wav', 'e649aa92_nohash_0.wav']
nine 	 ['38d78313_nohash_4.wav', 'bed06fac_nohash_1.wav', '7be9f58a_nohash_0.wav', '43b85b64_nohash_1.wav', '5de8f2f0_nohash_2.wav']
up 	 ['38d78313_nohash_4.wav', '27c30960_nohash_0.wav', 'e649aa92_nohash_0.wav', 'c08e5058_nohash_0.wav', '1890e5c3_nohash_2.wav']
right 	 ['38d78313_nohash_4.wav', '8fe52b97_nohash_1.wav', 'eee9b9e2_nohash_2.wav', '7be9f58a_nohash_0.wav', 'e649aa92_nohash_0.wav']
six 	 ['38d78313_nohash_4.wav', 'bed06fac_nohash_1.wav', '8335f893_nohash_1.wav', '7be9f58a_nohash_0.wav', '27c30960_nohash_0.wav']
wow 	 ['bed06fac_nohash_1.wav', '8fe52b97_nohash_1.wav', '43b85b64_nohash_1.wav', '6823565f_nohash_2.wav', 'f804cbb3_nohash_0.wav']
down 	 ['38d78313_nohash_4.wav', 'a40c62f1_nohash_2.wav', '8335f

In [120]:
import librosa
import librosa.display
from scipy import signal

def loadData(files):
  # Build Dataset
  new_rate = 8000
  xtrain, ytrain = [], []
  count = 0
  for label, filenames in files.items():
    if label not in ['', '_background_noise_']:
      for i, filename in enumerate(filenames[:50]):
        if i == 0:
          print(count)
          count += 1
        audio, new_rate = librosa.load(train_audio_path + label + '/' + filename, sr=new_rate)
        _, _, spectogram = log_specgram(audio, sample_rate=new_rate)
        
        xtrain.append(spectogram)
        ytrain.append(label)

        # mfccs = librosa.feature.mfcc(y=audio, sr=new_rate, n_mfcc = 50)
        # mfccs_processed = np.mean(mfccs.T,axis=0)
        
  return np.array(xtrain), np.array(ytrain)

def onehot(A, mapping=None):
  labels = set('yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown')
  
  if mapping is None:
    mapping = {}
    for i, label in enumerate(labels):
      temp = [0] * len(labels)
      temp[i] = 1
      mapping[label] = temp
    
  res = []
  for label in A:
    if label == '_background_noise_':
      res.append(mapping['silence'])
    elif label in labels:
      res.append(mapping[label])
    else:
      res.append(mapping['unknown'])
  
  return np.array(res), mapping

def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
    
def normalize(x):
  minX = np.amin(x)
  maxX = np.amax(x)
  gap = maxX - minX

  return (x - (minX + 0.5*gap)) / (0.5 * gap)

In [ ]:
X, Y = loadData(files)

In [ ]:
X = X.reshape(X.shape[0], 1, X[0].shape[0])
Y, mapping = onehot(Y)
X = normalize(xtrain)

xtrain, xval, ytrain, yval = train_test_split(X, Y, test_size=0.2, random_state=127)

print(A.shape, Y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Flatten

model = Sequential()
model.add(LSTM(32, return_sequences = True, input_shape = xtrain.shape[1:], activation = 'relu'))
model.add(LSTM(128, activation = 'relu'))

model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))

model.add(Dense(len(mapping), activation = 'softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(xtrain, ytrain, batch_size=16, validation_data=(xval, y_val), epochs=10, shuffle=True, verbose=1)